# Main calculations

Currently kick off by creating the `coeff_ser` pd.Series.  In due course might want to set up so this is automatically created when the library is imported (i.e. call the functions in the `__init__.py` of the module directory).  Though that would mean you had to have the target specified.  Not great.  Possibly best left as is - given that need target to be explicit.

## Next up
Just work through the component functions.
Haven't worked out how to do higher order calcs, eg for treatment impact or populations, but figure will need these components anyway.

In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from read_coeffs import *

In [3]:
wb = load_workbook('data/wsi_v0.1.31.xlsx', data_only=True)
coeff_ser = make_pdseries(get_coeffs(wb))

Can treat this series like an object (well it is an object, so..).  Eg:

In [7]:
coeff_ser.Prod_MCS_coeffs

0     1.0383
1     5.0122
2    32.5459
dtype: float64

In [8]:
def productivity(age, qol, debug=False):
    '''returns productivity, as % of time worked, for given age and QoL'''
    MCS = (coeff_ser.Prod_MCS_coeffs[0] * (age/10)) \
          + (coeff_ser.Prod_MCS_coeffs[1] * qol) \
            + coeff_ser.Prod_MCS_coeffs[2]

    PCS = (coeff_ser.Prod_PCS_coeffs[0] * (age/10)) \
          + (coeff_ser.Prod_PCS_coeffs[1] * qol) \
            + coeff_ser.Prod_PCS_coeffs[2]
    
    prod_sum = ((coeff_ser.Prod_prod_coeffs[0] * (age/10)) 
                + (coeff_ser.Prod_prod_coeffs[1] * ((age/10)**2)) 
                + (coeff_ser.Prod_prod_coeffs[2] * (MCS/10)) 
                + (coeff_ser.Prod_prod_coeffs[3] * ((MCS/10)**2)) 
                + (coeff_ser.Prod_prod_coeffs[4] * (PCS/10)) 
                + (coeff_ser.Prod_prod_coeffs[5] * ((PCS/10)**2)) 
                + coeff_ser.Prod_prod_coeffs[6])
    
    productivity = np.exp(prod_sum)/(1+np.exp(prod_sum))
    
    return productivity

In [42]:
def paid_prod(age=35, gen='F', qol=0.9, debug=False):
    '''X this new docstring here'''
#     print('productivity is ', productivity(age, qol, debug))
    if gen=='F':
        wage = coeff_ser.Prod_wagepcm_F_byage[age]    
    else:
        wage = coeff_ser.Prod_wagepcm_M_byage[age]
    on_costs = coeff_ser.on_costs
#     print("wage is ", wage)
#     print("on costs are ", on_costs)
    prod = productivity(age, qol, debug=False) 
#     print("productivity is ", prod)
    return prod * wage * (1 + on_costs)

In [43]:
timeit paid_prod(88, 'M', 0.5)

100 loops, best of 3: 4.04 ms per loop


In [43]:
coeff_ser.on_costs

0    1135.462241
dtype: float64